# Pyxem orientation mapping tutorial
pyXem (Python Crystallographic Electron Microscopy) is an open-source Python library for crystallographic electron microscopy. It builds on the tools for multi-dimensional data analysis provided by the HyperSpy library for treatment of experimental electron diffraction data and tools for atomic structure manipulation provided by the PyMatGen library.

This tutorial will cover data loading, simple operations including cropping, background preparation and removal, saving ED patterns as .txt files as well as peak enhancement and orientation mapping.

### Authors:
15/01/2018 Jędrzej Morzy
    
### Requirements:
Pyxem,
Hyperspy, 
pymatgen

## 1. Initial imports and data loading

In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import pyxem as pc
import numpy as np
import transforms3d
from skimage.morphology import erosion
from skimage.morphology import disk
from pymatgen.transformations.standard_transformations import RotationTransformation

#For pattern processing:
from scipy.ndimage import gaussian_filter
from skimage.filters import (threshold_sauvola, threshold_otsu)

Load the data from file, preview

In [2]:
dp_all = pc.load('Data_files/nanowire_precession.hdf5')
dp_all.set_calibration(0.035)

In [4]:
dp_all.plot()

## 2. Cropping the data set
Using a an interactive ROI (Region of Image) tool, choose the region to crop. Initial ROI is set to a 35x35 pixel region.

In [3]:
roi = pc.roi.RectangularROI(left=10, right=230, top=650, bottom=962.5)
dp_all.plot()
dp=roi.interactive(dp_all)
dp_all.set_calibration(0.035)
dp.plot()

It is possible to access last selected ROI details and view the cropped region if needed:

In [ ]:
roi

In [12]:
dp.plot()

## 3. Saving a test ED pattern to file
In the inav coordinates, choose which pattern you wish to save.

In [ ]:
test_ed_pattern = dp2.inav[13,12].data

In [ ]:
np.savetxt('test_ed_pattern.txt', test_ed_pattern, delimiter = "\t", newline="\r\n")

## 4. Pattern processing, peak enhancing

This section allows for the improvement in later pattern matching by removal of background from the patterns and enhancing the peaks through gaussian filter kernel.

What size of diffraction pattern is needed? (in pixels)

In [4]:
size = 80

### Definitions

In [5]:
def blur(a, sigma):
    a = gaussian_filter(a, sigma = sigma, mode = 'mirror')
    return a

def enhance(a, sigma):
    a2 = gaussian_filter(a, sigma = sigma, mode = 'constant', cval = 255)
    a = a - a2
    a[a < 0] = 0
    a = np.round(255*(a / np.max(a)))
    return a

def blur_crop (img, size, sigma):
    bl = gaussian_filter(img, sigma = sigma)
    peak = np.max(bl)
    centre_arr = np.where(bl == peak)
    a = int(size/2)
    startx = int(centre_arr[0][0] - a)
    starty = int(centre_arr[1][0] - a)
    return img[starty : starty+size, startx : startx+size]

def bg_blur_crop (img, size, sigma, radius):
    bl = gaussian_filter(img, sigma = sigma)
    peak = np.max(bl)
    centre_arr = np.where(bl == peak)
    a = (size/2)
    xc = centre_arr[0][0]
    yc = centre_arr[1][0]
    startx = int(xc - a)
    starty = int(yc - a)
    
    h = size/2
    # centre and radius of circle
    # x and y coordinates per every pixel of the image
    x, y = np.meshgrid(np.arange(144), np.arange(144))
    # squared distance from the center of the circle
    d2 = (x - xc)**2 + (y - yc)**2
    # mask is True inside of the circle
    mask = d2 > radius**2
#    img = img*mask
    return img[starty : starty+size, startx : startx+size]

def pre_process(z, bg, size, sigma_blur, sigma_enhance, k, window_size):
#     Good starting values:
#     sigma_blur = 1.6
#     sigma_enhance = 0.5
#     threshold = 6.5
#     window_size = 11
#     k = 0.01
 #   image = z
    image = blur_crop(z, size = size, sigma = 3)
#    bg = blur_crop(bg, size = size) <- done beforehand, check above cells, remember about size to be the same as in image blur_crop
    image = image - bg
    for i in range (0,image.shape[0]):
        for j in range (0,image.shape[1]):
            if image[i,j]<0:
                image[i,j]=0
                        
    image_blur = blur(image, sigma = sigma_blur)
    image = enhance(image_blur, sigma = sigma_enhance)
    blurred = blur(image, sigma = 3)
    mask = blurred > 0.3
    image = image*mask
    thresh_sauvola = threshold_sauvola(image, window_size=window_size, k=k)
    binary_sauvola = image > thresh_sauvola
    binary_sauvola_blur = gaussian_filter(binary_sauvola, sigma = 0.25)
    final = image*binary_sauvola_blur
    for i in range (0,image.shape[0]):
        for j in range (0,image.shape[1]):
            if final[i,j]>10:
                final[i,j]=10 
    final = gaussian_filter(final, sigma = 1.5)

    
    return final

#### Preparation of background:

Crop just the vacuum area of the sample in order to create an average background pattern that will be subtracted from all of the patterns

In [6]:
roi2 = pc.roi.RectangularROI(left=10, right=50, top=1050, bottom=1212.5)
dp_all.plot()
dp_vac=roi2.interactive(dp_all)
dp_vac.plot()

Average the patterns in cropped region and plot them.

In [7]:
dp_bg = dp_vac.mean((0,1))
dp_bg.plot()

In [8]:
dp_bg_blurred = blur(dp_bg.data, sigma =2)
imgplot = plt.imshow(dp_bg_blurred)

In [9]:
dp_bg2 = bg_blur_crop(dp_bg.data, size = size, sigma = 10, radius = 6.6)
imgplot=plt.imshow(dp_bg2)

#### Peak enhancement function
TO Do: Work on a copied set of data, so they can be later compared. First look into the function itself for an explanation of how it works and what are the parameters needed.

In [11]:
dp2 = dp.copy()
dp2.set_calibration(0.035)
dp2.map(pre_process, bg = dp_bg2, size = size, sigma_blur = 1.6, sigma_enhance = 6.5,k = 0.001, window_size = 11)

A Jupyter Widget

Check how it went with the set parameters and compare with the original

In [12]:
dp2.plot()

In [19]:
dp.plot()

## 5. Virtual dark field images

It is possible to create virtual dark field (VDF) images from original data. It works by choosing a virtual aperture on the diffraction pattern and plotting the image using intensity from just the virtual aperture region for each pixel. 

In [ ]:
circ = pc.roi.CircleROI(cx=0, cy=0, r=0.1)
dp.plot_interactive_virtual_image(circ)

If you want to save the image with current position of the circle:

In [ ]:
vdf=dp.get_virtual_image(circ)

Save and load to/from file

In [ ]:
vdf=vdf.as_signal2D((0,1))
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

In [ ]:
vdfload = pc.load('vdfeg.tif')
vdfload.plot()

## 6. Pattern matching and orientation mapping

Import structure information from a .cif file


In [13]:
structure = pc.Structure.from_file("Data_files/GaAs.cif")

Create a rotation list from file. The file itself is an array of unequivalent rotations expressed as sets of three Euler angles (Bunge ZXZ convention).

In [14]:
rot_array = np.loadtxt('m-3m_grid_euler')

Create structure library based on imported .cif file and the rotations to later match the image with the library

In [16]:
edc = pc.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pc.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib["CsPbBr3"] = (structure, rot_array)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=0.035,
                                            reciprocal_radius=1.,
                                            representation='euler')

Perform correlation of each of the experimental diffraction patterns with every possible pattern (simulated, in the library). The function chooses 5 best results for each pixels and saves all the details in an array.

In [23]:
from pycrystem.indexation_generator import IndexationGenerator

In [24]:
indexer = IndexationGenerator(dp2, library)

In [25]:
match_results = indexer.correlate(parallel = True)

A Jupyter Widget

C:\Users\jdkm2\Anaconda3\lib\site-packages\pycrystem-0.4-py3.6-win-amd64.egg\pycrystem\utils\__init__.py:118: RuntimeWarning: invalid value encountered in double_scalars
  np.sqrt(np.dot(intensities_2, intensities_2))


Resulting array has the following structure: data for each pixel of the image has columns: 1 = phase id column 2-4 = Euler angles in the zxz convention (in radians) column 5 = correlation scores associated with the orientation and phase (top 5 for each phase, maximum value = 1)

Look into the details of patterns matched to at x=15, y=15 pixel:

In [26]:
match_results.inav[15,15].data[0:5]

array([[ 0.        ,  5.321473  ,  0.741765  ,  1.659844  ,  0.27681148],
       [ 0.        ,  4.855189  ,  0.654498  ,  1.951595  ,  0.2740841 ],
       [ 0.        ,  4.552104  ,  0.741765  ,  1.469282  ,  0.27320845],
       [ 0.        ,  5.57793   ,  0.741765  ,  0.09439   ,  0.27318536],
       [ 0.        ,  1.713596  ,  0.654498  ,  5.093188  ,  0.27317415]])

Obtain a 'final' crystallographic map by returning the best matching phase and orientation for each pixel.

The crystallographic map results are:

column 1 = phase id column 2-4 = Euler angles column 5 = correlation score column 6 = difference between this correlation score and the 2nd best correlation score

In [29]:
cryst_map = match_results.get_crystallographic_map()

A Jupyter Widget

array([  0.00000000e+00,   2.85599000e-01,   3.05433000e-01,
         3.25266000e-01,   9.99853279e-01,   1.74873927e-05])

Get an orientation map.
An alternative method with color-coded orientations can be done by MTEX MatLab library. For that matching results must be exported into a file that can be read by MTEX.

http://mtex-toolbox.github.io/

In [31]:
cryst_map.get_orientation_image().plot()

A Jupyter Widget

## 7. Saving match results to file for MTEX

Create an array where column 1 = phase id column, 2-4 = Euler angles column, 5 = correlation score column, 6 and 7 = x and y

In [32]:
results_array = np.zeros([0,7])
for i in range (0, match_results.data.shape[1]):
    for j in range (0, match_results.data.shape[0]):
        newrow = match_results.inav[i,j].data[0,0:5]
        newrow = np.append(newrow, [i,j])
        results_array = np.vstack([results_array, newrow])

Save to a txt file

In [33]:
np.savetxt('gaas_test2.txt', results_array, delimiter = "\t", newline="\r\n")

## 8. Checking the quality of matching

It is possible to check what pattern has been matched to specific pattern in the experimental data.
X and Y are the coordinates of the pattern to be checked. Extracting the Euler angle rotation for the specific pattern:

In [ ]:
x = 5
y = 3
euler_angles = match_results.inav[x,y].data[0,1:4]

Simulate the diffraction pattern using Euler angles specified above, extract a single pattern from the data

In [45]:
rot = euler_angles
edc = pc.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pc.DiffractionLibraryGenerator(edc)
single_pattern_lib = dict()
single_pattern_lib["GaAs"] = (structure, rot)
simulated_pattern= diff_gen.get_diffraction_library(single_pattern_lib,
                                            calibration=0.032,
                                            reciprocal_radius=1.,
                                            representation='euler')
exp_pattern = dp2.inav[13,12].data

In [46]:
library_test.plot()
exp_pattern.plot()